In [1]:
import torchtext
import pandas as pd
import numpy as np
import sklearn.metrics
from collections import defaultdict
from tqdm import tqdm

In [2]:
df=pd.read_csv('./parallelograms-revisited/experiment1_data_completions/experiment1a_counts.csv')

In [3]:
df=df.drop(columns=['prompt'])

In [4]:
vocab = torchtext.vocab.GloVe(name='840B', dim=300)

In [10]:
def get_ranks(df, cd_ranks, p_ranks):
    for idx, row in tqdm(df.iterrows()):
    # for idx, row in df.iterrows():
        w1, w2, w3 = row['prompt_w1'], row['prompt_w2'], row['prompt_w3']
        # print(w1, w2, w3)

        if w1 not in vocab.stoi or w2 not in vocab.stoi or w3 not in vocab.stoi:
            print(w1, w2, w3)
            continue
            
        a = vocab[w1]
        b = vocab[w2]
        c = vocab[w3]
        
        row.pop('prompt_w1')
        row.pop('prompt_w2')
        row.pop('prompt_w3')
        
        # print('popped')

        cd_sim = sklearn.metrics.pairwise.cosine_similarity(c.reshape(1, -1), vocab.vectors)
        cd_sorted_indices = (-cd_sim).argsort()[0]
        cd_stoi = dict(zip(vocab.itos, cd_sorted_indices))

        d = b - a + c 
        p_sim = sklearn.metrics.pairwise.cosine_similarity(d.reshape(1, -1), vocab.vectors)
        p_sorted_indices = (-p_sim).argsort()[0]
        p_stoi = dict(zip(vocab.itos, p_sorted_indices))
        
        # print('constructed stoi')
        
        for key, value in row.items():
            if key not in cd_stoi:
                continue
            cd_idx = cd_stoi[key]
            cd_ranks[cd_idx] += value

            p_idx = p_stoi[key]
            p_ranks[p_idx] += value

In [ ]:
cd_ranks = np.zeros(len(vocab.itos))
p_ranks = np.zeros(len(vocab.itos))
get_ranks(df, cd_ranks, p_ranks)

25it [04:06,  9.87s/it]

lamb chop lamb chapter


31it [04:56,  9.51s/it]

launchpad helicopter diving board


34it [05:15,  8.65s/it]

...

43it [06:24,  9.34s/it]

basket picnic lunch box


45it [06:34,  8.03s/it]

furnace coal wood stove


55it [08:04,  9.77s/it]

In [ ]:
cd_cumsum = np.cumsum(cd_ranks)
p_cumsum = np.cumsum(p_ranks)

In [ ]:
cd_cumsum, p_cumsum